In [1]:
import os
os.system("cls")
print("Loading Payroll Program...")
import pysftp
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import time
from time import sleep
import shutil 
import sys
from tqdm import tqdm
from dotenv import load_dotenv
import pdfkit
import warnings
warnings.filterwarnings("ignore")
now = datetime.datetime.now()
timestamp = str(now.strftime("%Y%m%d_%H%M%S"))

def taxes_single(income):
        if income < 9950:
            tax = income * 0.1
        elif income > 9951 and income <= 40525:
            tax = ((income-9950)*(0.12)) + (9950 * 0.1)
        elif income > 40526 and income <= 86375:
            tax = ((income-40525)*(0.22)) + (9950 * 0.1) + ((40525 - 9950) * 0.12)
        elif income > 86376 and income <= 164925:
            tax = ((income-86375)*(0.24)) + (9950 * 0.1) + ((40525 - 9950) * 0.12) + ((86376 - 40525 - 9950) * 0.22)
        elif income > 164926 and income <= 209425:
            tax = ((income-164925)*(0.32)) + (9950 * 0.1) + ((40525 - 9950) * 0.12) + ((86376 - 40525 - 9950) * 0.22) + ((164925 - 86375 - 40525 - 9950) * 0.24)
        elif income > 209426 and income <= 523600:
            tax = ((income-209425)*(0.35)) + (9950 * 0.1) + ((40525 - 9950) * 0.12) + ((86376 - 40525 - 9950) * 0.22) + ((164925 - 86375 - 40525 - 9950) * 0.24) + ((209425 - 164925 - 86375 - 40525 - 9950) * 0.32)
        elif income > 523601:
            tax = ((income-523600)*(0.37)) + (9950 * 0.1) + ((40525 - 9950) * 0.12) + ((86376 - 40525 - 9950) * 0.22) + ((164925 - 86375 - 40525 - 9950) * 0.24) + ((209425 - 164925 - 86375 - 40525 - 9950) * 0.32) + ((523600 - 209425 - 164925 - 86375 - 40525 - 9950) * 0.35)
        return (tax / 52)

def taxes_married(income):
        if income < 19900:
            tax = income * 0.1
        elif income > 19901 and income <= 81050:
            tax = ((income-19900)*(0.12)) + (19900 * 0.1)
        elif income > 81051 and income <= 172750:
            tax = ((income-81050)*(0.22)) + (19900 * 0.1) + ((81050 - 19900) * 0.12)
        elif income > 172751 and income <= 329850:
            tax = ((income-172750)*(0.24)) + (19900 * 0.1) + ((81050 - 19900) * 0.12) + ((172751 - 81050 - 19900) * 0.22)
        elif income > 329851 and income <= 418850:
            tax = ((income-329850)*(0.32)) + (19900 * 0.1) + ((81050 - 19900) * 0.12) + ((172751 - 81050 - 19900) * 0.22) + ((329850 - 172750 - 81050 - 19900) * 0.24)
        elif income > 418851 and income <= 628300:
            tax = ((income-418850)*(0.35)) + (19900 * 0.1) + ((81050 - 19900) * 0.12) + ((172751 - 81050 - 19900) * 0.22) + ((329850 - 172750 - 81050 - 19900) * 0.24) + ((418850 - 329850 - 172750 - 81050 - 19900) * 0.32)
        elif income > 628301:
            tax = ((income-628300)*(0.37)) + (19900 * 0.1) + ((81050 - 19900) * 0.12) + ((172751 - 81050 - 19900) * 0.22) + ((329850 - 172750 - 81050 - 19900) * 0.24) + ((418850 - 329850 - 172750 - 81050 - 19900) * 0.32) + ((628300 - 418850 - 329850 - 172750 - 81050 - 19900) * 0.35)
        return (tax / 52)

def medicare(income):
        medicare_tax = income * 0.0145
        return medicare_tax

def ss(income):
        ss_tax = income * 0.0620
        return ss_tax

os.system("cls")
print("This is a Payroll Processing Program designed to retrieve information from the employee self service portal, validate / update data to Employee Master Records and process payroll with ACH file output")
os.system("pause")

In [169]:
#Creating Employee Master Backup<final>
print("Creating Employee Master Backup...") 
for i in tqdm(range(10)):
    sleep(0.5)
src = Path("../Resources/Source_data/master.csv")
dst = Path("../Resources/Source_data/Archive/MASTER_BACKUP.csv")
shutil.copy(src, dst)
print("Employee Master Backup Complete")

#Downloading Updates
# load_dotenv()
# server = os.getenv("ftp_server")
# username = os.getenv("ftp_user")
# password = os,getenv("ftp_pass")
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys = None
# sftp = pysftp.Connection(server, username=username, password=password, cnopts=cnopts)
print("Downloading Data...")
# sftp.get('/outbound/noonah_CHalling/updates.csv')
for i in tqdm(range(10)):
    sleep(1)
print("Download Complete")
os.system("pause")

  0%|          | 0/10 [00:00<?, ?it/s]

Creating Employee Master Backup...


  0%|          | 0/10 [00:00<?, ?it/s]

Employee Master Backup Complete


100%|██████████| 10/10 [00:10<00:00,  1.01s/it]

Download Complete


0

In [170]:
#Validating Employee Update Data<final>
os.system("cls")
print("Validating Employee Update Data...")
upval_employee_data_updates = pd.read_csv(Path("../Resources/Source_data/updates.csv"), dtype = {"EMP_NO" : str, "SSN" : str, "BANK_RTN" : str, "BANK_ACC" : str}) 

upval_employee_data_updates["Issue"] = np.where(upval_employee_data_updates["EMP_NO"].isna(), "Employee Number is Missing",
    np.where(upval_employee_data_updates["FIRST_NAME"].isna(), "Name is Missing",
    np.where(upval_employee_data_updates["SSN"].isna(), "SSN is Missing",
    np.where(upval_employee_data_updates["JOBTITLE"].isna(), "Job Title is Missing",
    np.where(upval_employee_data_updates["HIREDATE"].isna(), "Hire Date is Missing",
    np.where(upval_employee_data_updates["DOB"].isna(), "Date of Birth is Missing",
    np.where(upval_employee_data_updates["FEDFILINGSTATUS"].isna(), "Federal Filing Status is Missing",
    np.where(upval_employee_data_updates["WORKSTATE"].isna(), "Work State is Missing",
    np.where(upval_employee_data_updates["BANK_RTN"].isna(), "Bank Routing Number is Missing",
    np.where(upval_employee_data_updates["BANK_ACC"].isna(), "Bank Account Number is Missing",
    np.where(upval_employee_data_updates["EMP_STATUS"].isna(), "Employment Status is Missing",
    np.where(upval_employee_data_updates["PAYCODE"].isna(), "Pay code is Missing",
    np.where(upval_employee_data_updates["PAYRATE"].isna(), "Pay rate is Missing","")))))))))))))

upval_employee_data_good_records = upval_employee_data_updates.dropna(subset=["FIRST_NAME","SSN","JOBTITLE","HIREDATE","DOB","FEDFILINGSTATUS","WORKSTATE","BANK_RTN","BANK_ACC","EMP_STATUS","PAYCODE","PAYRATE"])
upval_employee_data_good_records = upval_employee_data_good_records.drop(labels="Issue", axis=1)

upval_employee_data_issues = upval_employee_data_updates[upval_employee_data_updates["Issue"] != ""]
upval_employee_data_issues = upval_employee_data_issues.rename_axis(mapper="Record#", axis=1)
upval_employee_data_issues.index+= 1
upval_employee_data_bad_records = upval_employee_data_issues
upval_employee_data_bad_records.to_csv(Path("../Resources/Source_data/badrec.csv"), index=False, na_rep="")
upval_employee_data_issues_report = upval_employee_data_issues[["Issue"]]
for i in tqdm(range(10)):
    sleep(1)
print("Validation Complete")
ques1 = input("Do you want to review (Y / N)?  ")
if ques1 == "Y": 
    print(upval_employee_data_issues_report)
    os.system("pause")
    os.system("cls")
elif ques1 == "N": 
    os.system("cls")
else: 
    print("Please enter Y or N.") 


  0%|          | 0/10 [00:00<?, ?it/s]

Validating Employee Update Data...


100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


Validation Complete
Record#            Issue
1        Name is Missing
5         SSN is Missing


In [171]:
#Updating Employee Master<final>
print("Adding Good Records to Employee Master")
upval_employee_data = pd.read_csv(Path("../Resources/Source_data/master.csv"), index_col = "EMP_NO", keep_default_na = False, dtype = {"SSN" : str, "BANK_RTN" : str, "BANK_ACC" : str})
upval_employee_data_updates = pd.read_csv(Path("../Resources/Source_data/updates.csv"), index_col = "EMP_NO", keep_default_na = False, dtype = {"SSN" : str, "BANK_RTN" : str, "BANK_ACC" : str}) 
upval_employee_data = upval_employee_data[~upval_employee_data.index.isin(upval_employee_data_updates.index)].reset_index()
upval_employee_data_updates = upval_employee_data_updates.reset_index()
upval_employee_data = upval_employee_data.append(upval_employee_data_updates, ignore_index="True", verify_integrity="True").sort_values(by="EMP_NO").astype("str")
upval_employee_data.to_csv(Path("../Resources/Source_data/master1.csv"), index=False)
for i in tqdm(range(10)):
    sleep(1)
print("Update Complete")
os.system("pause")
os.system("cls")

  0%|          | 0/10 [00:00<?, ?it/s]

Adding Good Records to Employee Master


100%|██████████| 10/10 [00:10<00:00,  1.01s/it]

Update Complete


0

In [172]:
#Loading Employee Data
employee_data = pd.read_csv(Path("../Resources/Source_data/master1.csv"), header = 0, usecols = ["EMP_NO","FEDFILINGSTATUS","EMP_STATUS","PAYCODE","PAYRATE"])
hour_data = pd.read_csv(Path("../Resources/Source_data/hours.csv")) 
employee_ach_data = pd.read_csv(Path("../Resources/Source_data/master1.csv"), header = 0, usecols = ["EMP_NO","FIRST_NAME","LAST_NAME","BANK_RTN","BANK_ACC"], dtype = {"BANK_RTN" : str, "BANK_ACC" : str})

In [173]:
#Calculating Hourly Wages
hourly_emp_data = employee_data.loc[employee_data["PAYCODE"] == "Hourly"]
hourly_emp_data = employee_data.loc[employee_data["EMP_STATUS"] == "Active"]
hourly_grid = pd.merge(hourly_emp_data, hour_data, how = "inner", on = "EMP_NO")
hourly_grid["WAGES"] = np.where(hourly_grid["HOURS"] > 40, 
    ((hourly_grid["HOURS"]-40)*1.5*hourly_grid["PAYRATE"])+ (hourly_grid["PAYRATE"]*40), 
    (hourly_grid["HOURS"] * hourly_grid["PAYRATE"]))
# hourly_grid.drop(["EMP_STATUS","PAYCODE","PAYRATE","HOURS"], axis=1).round(2)

#Calculating Salary Wages
salary_grid = employee_data.loc[employee_data["PAYCODE"] == "Salary"]
salary_grid = salary_grid.loc[salary_grid["EMP_STATUS"] == "Active"]
salary_grid["WAGES"] = salary_grid["PAYRATE"] / 52
salary_grid.drop(["EMP_STATUS"], axis=1).round(2)

#Combining Salary and Hourly Wages
all_wages = hourly_grid.append(salary_grid, ignore_index=True, verify_integrity= True)
all_wages.drop(["EMP_STATUS","PAYRATE","HOURS"], axis=1).round(2)

#Tax Calculation
all_wages["WAGESANA"] = all_wages["WAGES"] * 52
tax_single_all_wages = all_wages.loc[all_wages["FEDFILINGSTATUS"] == "Single or Married filing separately"]
tax_single_all_wages["TAX"]=tax_single_all_wages["WAGESANA"].apply(taxes_single)
tax_single_all_wages["NET_WAGES"]= tax_single_all_wages["WAGES"] - tax_single_all_wages["TAX"]

tax_married_all_wages = all_wages.loc[all_wages["FEDFILINGSTATUS"] == "Married or filing jointly"]
tax_married_all_wages["TAX"]=tax_married_all_wages["WAGESANA"].apply(taxes_single)
tax_married_all_wages["NET_WAGES"]= tax_married_all_wages["WAGES"] - tax_married_all_wages["TAX"]

final_wages = tax_single_all_wages.append(tax_married_all_wages, ignore_index=True, verify_integrity= True)
final_wages.drop(["EMP_STATUS","PAYRATE","HOURS","WAGESANA"], axis=1).round(2)

,EMP_NO,FEDFILINGSTATUS,PAYCODE,WAGES,TAX,NET_WAGES
0,101001,Single or Married filing separately,Hourly,847.50,104.69,742.81
1,101005,Single or Married filing separately,Hourly,1800.00,274.93,1525.07
2,101006,Single or Married filing separately,Hourly,1062.50,151.99,910.51
3,101009,Single or Married filing separately,Hourly,600.00,68.17,531.83
4,101011,Single or Married filing separately,Hourly,1400.00,226.24,1173.76
5,101000,Single or Married filing separately,Salary,2307.69,396.77,1910.92
6,101004,Single or Married filing separately,Salary,1442.31,235.55,1206.76
7,101008,Single or Married filing separately,Salary,1826.92,281.39,1545.53
8,101003,Married or filing jointly,Hourly,1400.00,226.24,1173.76


In [174]:
#Creating ACH File
# employee_ach_data["NAME"] = employee_ach_data["FIRST_NAME"].str.upper() + [" "].str + employee_ach_data["LAST_NAME"].str.upper()
employee_ach_data = pd.merge(final_wages, employee_ach_data, how="inner", on = "EMP_NO")
employee_ach_data = employee_ach_data.drop(["FEDFILINGSTATUS","WAGES","WAGESANA","TAX","EMP_STATUS","PAYCODE", "PAYRATE", "HOURS"], axis=1).round(2)
employee_ach_data

,EMP_NO,NET_WAGES,FIRST_NAME,LAST_NAME,BANK_RTN,BANK_ACC
0,101001,742.81,Jamel,Bardney,124085244,1321541313
1,101005,1525.07,Thomas,Hendricks,073905912,2136564123
2,101006,910.51,Marcell,Mitchell,124085244,16848643139846
3,101009,531.83,Dewayne,Chester,124085244,569746414
4,101011,1173.76,Malik,Charleston,256074974,1235483141
5,101000,1910.92,Eric,Lane,256074974,1235483141
6,101004,1206.76,Terano,McCann,103100195,1564642132
7,101008,1545.53,James,Wimbush,124085244,516461313
8,101003,1173.76,Ivonne,Peralta Velazquez,091000019,21546931354
